# Wikipedia-API

In [1]:
import os
import string
import re
import warnings
warnings.filterwarnings("ignore")
!pip install git+https://github.com/boudinfl/pke.git
import pke

  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-azlfb5iv
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-azlfb5iv
  Resolved https://github.com/boudinfl/pke.git to commit 5af1f817e0211c33ac3f90e1e86bb5c1283448e8
  Preparing metadata (setup.py) ... - done
  Created wheel for pke: filename=pke-2.0.0-py3-none-any.whl size=6160276 sha256=574e7c35dcaa1ebbd8c342b8e5d2d45ee59c7fc2a6ecdc662dda004383de3567
  Stored in directory: /tmp/pip-ephem-wheel-cache-hk23rap9/wheels/fa/b3/09/612ee93bf3ee4164bcd5783e742942cdfc892a86039d3e0a33
Successfully built pke
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency confli

In [2]:
!pip install Wikipedia-API

import wikipediaapi

wiki_wiki = wikipediaapi.Wikipedia('en')

ml_art = wiki_wiki.page('Deep_Learning')

#print("Page - Exists: %s" % ml_art.exists())

#print("Page - Title: %s" % ml_art.title)

#print("Page - Summary: %s" % ml_art.summary[0:60])

#print(ml_art.fullurl)

ml_ftxt = ml_art.text

  Preparing metadata (setup.py) ... - done
  Created wheel for Wikipedia-API: filename=Wikipedia_API-0.5.4-py3-none-any.whl size=13477 sha256=cea06a164511bc1e2cb406815466d6623b954019c60c78db69f5191a321b9a3a
  Stored in directory: /root/.cache/pip/wheels/d3/24/56/58ba93cf78be162451144e7a9889603f437976ef1ae7013d04
Successfully built Wikipedia-API


# print sample of text

In [3]:
ml_ftxt[:2000]

'Deep learning  (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised.Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks,  convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.Artificial neural networks (ANNs) were inspired by information processing and distributed communication nodes in biological systems. ANNs have various differences from biological brains.  Specifically, artificial neural networks tend to

# perform some cleaning on the text

In [4]:
import re 
ml_ftxt= re.sub(r'\n', " ", ml_ftxt)
ml_ftxt = ml_ftxt.translate(str.maketrans(' ', ' ','!"#$%&\'()*+-/:;<=>?@[\\]^_`{|}~'))
ml_ftxt = re.sub(r'[A-Za-z0-9]*@[A-Za-z]*\.?[A-Za-z0-9]*',"",ml_ftxt )

# Print sample of text after cleaning

In [5]:
ml_ftxt[:2000]

'Deep learning  also known as deep structured learning is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semisupervised or unsupervised.Deeplearning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks,  convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.Artificial neural networks ANNs were inspired by information processing and distributed communication nodes in biological systems. ANNs have various differences from biological brains.  Specifically, artificial neural networks tend to be st

# TfIdf

In [6]:
extractor = pke.unsupervised.TfIdf()
extractor.load_document(input='rm')
extractor.candidate_selection() 
extractor.candidate_weighting()
keyphrases = extractor.get_n_best(n=10)
extractor = pke.unsupervised.TopicRank()
extractor.load_document(input=ml_ftxt, language='en')

In [7]:
keyphrases = extractor.get_n_best(n=5, stemming=False)
for i, (candidate, score) in enumerate(keyphrases):
    print("rank {}: {} ({})".format(i, candidate, score))

biglist = []
def alllist (var):
    for i in var:
        biglist.append(i[0])

# TopicRank

In [8]:
grammar="NP: {<ADJ>*<NOUN|PROPN>+}"
extractor = pke.unsupervised.TopicRank()
extractor.load_document(input=ml_ftxt, language='en')
extractor.grammar_selection(grammar=grammar)
extractor.candidate_weighting()
keyphrases = extractor.get_n_best(n=5, stemming=True)

top5 = [candidate for candidate, weight in keyphrases]
print("top-5 keyphrases:", '; '.join(top5))
alllist (keyphrases)

top-5 keyphrases: artifici neural network; deep structur learn; multipl layer; imag; data


# KPMiner

In [9]:
list2=[]
extract = pke.unsupervised.KPMiner()
extract.load_document(input=ml_ftxt, language='en')
extract.candidate_selection()
extract.candidate_weighting()
keyphrases = extract.get_n_best(n=10)
list2.append(keyphrases)
alllist (keyphrases)

In [10]:
ss= set(biglist)
for i in ss :
    print(i , biglist.count(i))

speech 1
deep neural networks 1
machine learning 1
deep structur learn 1
data 1
neural networks 1
imag 1
artifici neural network 1
deep neural 1
multipl layer 1
deep learning 1
neural 1
speech recognition 1
deep 1
artificial neural 1


# Map each important word to the paragraphs found in text

In [11]:
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor

def tokenize_sentences(text):
    sentences = [sent_tokenize(text)]
    sentences = [y for x in sentences for y in x]
    # Remove any short sentences less than 20 letters.
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values
    return keyword_sentences

sentences = tokenize_sentences(ml_ftxt)
keyword_sentence_mapping = get_sentences_for_keyword(ss, sentences)

In [12]:
sentences  = list(keyword_sentence_mapping.values())
sentences = sentences[10]

In [13]:
#final_sentences = []
#for i in sentences:
#    final_sentences.append("".join(i))
#sentences = final_sentences 

In [14]:
!pip install -U transformers==3.0.0
!python -m nltk.downloader punkt
!git clone https://github.com/patil-suraj/question_generation.git
%cd question_generation
!pip install python-pipeline
from pipelines import pipeline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.6/754.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 14.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.20.1
    Uninstalling transformers-4.20.1:
      Successfully uninstalled transformers-4.20.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.10.0 requires protobuf==3.20.0, but you have protobuf 3.19.4 which is incompatible.
allennlp 2.10.0 requires transformers<4.21,>=4.1, but you have transformers 3.0.0 which is incompatible.
/opt/conda/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of packa

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)


# Single task QA

In [15]:
nlp = pipeline("question-generation")

Downloading:   0%|          | 0.00/627 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [16]:
five_Questions1 = []
for i in range(0,5):
    five_Questions1.append(nlp(sentences[i]))

In [17]:
five_Questions1 = five_Questions1[2:]

In [18]:
five_Questions1

[[{'answer': 'artificial neural networks',
   'question': 'What are most modern deep learning models based on?'},
  {'answer': 'convolutional neural networks CNNs',
   'question': 'What are most modern deep learning models based on?'}],
 [{'answer': 'artificial neural networks',
   'question': 'What are most modern deep learning models based on?'},
  {'answer': 'convolutional neural networks CNNs',
   'question': 'What are most modern deep learning models based on?'}],
 [{'answer': 'trophicfactors',
   'question': 'What waves do infants brain seem to organize themselves under?'}]]

# Multitask QA-QG

In [19]:
nlp = pipeline("multitask-qa-qg")

In [20]:
five_Questions2 = []
for i in range(0,5):
    five_Questions2.append(nlp(sentences[i]))

In [21]:
five_Questions2 = five_Questions2[2:]

In [22]:
five_Questions2

[[{'answer': 'artificial neural networks',
   'question': 'What are most modern deep learning models based on?'},
  {'answer': 'convolutional neural networks CNNs',
   'question': 'What type of neural networks are most modern deep learning models based on?'}],
 [{'answer': 'artificial neural networks',
   'question': 'What are most modern deep learning models based on?'},
  {'answer': 'convolutional neural networks CNNs',
   'question': 'What type of neural networks are most modern deep learning models based on?'}],
 [{'answer': 'trophicfactors',
   'question': 'What is the influence of waves in the infants brain?'}]]

# Print the answers of the first 3 questions

In [23]:
five_Questions2

[[{'answer': 'artificial neural networks',
   'question': 'What are most modern deep learning models based on?'},
  {'answer': 'convolutional neural networks CNNs',
   'question': 'What type of neural networks are most modern deep learning models based on?'}],
 [{'answer': 'artificial neural networks',
   'question': 'What are most modern deep learning models based on?'},
  {'answer': 'convolutional neural networks CNNs',
   'question': 'What type of neural networks are most modern deep learning models based on?'}],
 [{'answer': 'trophicfactors',
   'question': 'What is the influence of waves in the infants brain?'}]]

In [24]:
x = 0
for i in five_Questions2:
        print(nlp({"question": i[0]["question"],"context": sentences[x]}))
        x = x+1

CNNs and LSTMs
raw features
abstract and composite representation


In [25]:
ml_ftxt

'Deep learning  also known as deep structured learning is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semisupervised or unsupervised.Deeplearning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks,  convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.Artificial neural networks ANNs were inspired by information processing and distributed communication nodes in biological systems. ANNs have various differences from biological brains.  Specifically, artificial neural networks tend to be st